In [7]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 395.18 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_9_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [11]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10,20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1035691, Wed Apr 15 14:03:52 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 4.036520824858413e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 4.050966609432511e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40665428.85183255] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40904636.64050346] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 4.033539229832682e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   28    532 3.842669348934161e+07 1.1e+00 2.02e+00  2e+00  2e+00 0:03.5
   60   1140 3.776326652458376e+07 1.2e+00 2.00e+00  2e+00  2e+00 0:07.6
  100   1900 3.684898763385800e+07 1.2e+00 1.69e+00  2e+00  2e+00 0:12.7
  149   2831 3.012362388022257e+07 1.2e+00 1.19e+00  1e+00  1e+00 0:18.7
  200   3800 1.766817303490267e+07 1.2e+00 7.33e-01  7e-01  7e-01 0:25.0
  265   5035 2.860762128842473e+06 1.2e+00 3.68e-01  4e-01  4e-01 0:33.1
  300   5700 2.902181762646735e+05 1.3e+00 2.50e-01  2e-01  3e-01 0:37.5
  379   7201 -2.113704692486063e+06 1.3e+00 1.01e-01  1e-01  1e-01 0:47.6
  400   7600 -2.335508560124665e+06 1.3e+00 7.89e-02  8e-02  8e-02 0:50.2
  496   9424 -2.561026211851090e+06 1.3e+00 2.62e-02  3e-02  3e-02 1:02.2
  500   9500 -2.564259917393014e+06 1.3e+00 2.49e-02  2e-02  3e-02 1:02.7
  600  11400 -2.589518161403343e+06 1.3e+00 7.95e-03  8e-03  8e-03 1:15.1
  700  13300 -2.592273919417158e+06 1.3e+00 2.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40597407.50487186] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

termination on tolfun=1e-11 (Wed Apr 15 14:27:29 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [-4.30954047  2.569323    3.08265617  1.01897069  1.08994452  4.07863229
  1.6860853  -1.0588295  ...]
std deviations: [2.43498342 2.43496211 2.43507484 2.43490971 2.43520231 2.4349763
 2.43468207 2.43535469 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1054562, Wed Apr 15 14:27:29 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 3.813922667280249e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 3.865021585948709e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

termination on tolfun=1e-11 (Wed Apr 15 21:07:41 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [-2.89298585 -3.73703028  2.95675644  4.29970215  4.99999897 -3.36704203
  3.96167685 -4.50195701 ...]
std deviations: [2.43514822 2.43552085 2.43595288 2.43531997 2.43538611 2.43566361
 2.43541365 2.43520861 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1129432, Wed Apr 15 21:07:41 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 3.483081526086731e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 3.992012049124414e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [44537591.22579077] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [44174258.03434558] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 4.144939826781331e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 1.944142420246965e+07 1.1e+00 1.91e+00  2e+00  2e+00 0:03.5
   68   1292 6.596219656074286e+06 1.1e+00 1.59e+00  2e+00  2e+00 0:07.5
  100   1900 -2.655181446995776e+06 1.2e+00 1.34e+00  1e+00  1e+00 0:11.1
  155   2945 -1.576792077836256e+07 1.2e+00 9.40e-01  9e-01  9e-01 0:17.2
  200   3800 -2.522642406635225e+07 1.2e+00 6.69e-01  7e-01  7e-01 0:22.2
  272   5168 -3.173436750299334e+07 1.2e+00 3.79e-01  4e-01  4e-01 0:30.2
  300   5700 -3.306731472792939e+07 1.3e+00 3.07e-01  3e-01  3e-01 0:33.3
  391   7429 -3.552658426280540e+07 1.3e+00 1.65e-01  2e-01  2e-01 0:43.4
  400   7600 -3.564020702978220e+07 1.3e+00 1.54e-01  2e-01  2e-01 0:44.4
  500   9500 -3.630683180142923e+07 1.3e+00 8.32e-02  8e-02  8e-02 0:55.4
  600  11400 -3.670088417843761e+07 1.5e+00 6.27e-02  6e-02  6e-02 1:06.5
  700  13300 -3.705762188590380e+07 1.9e+00 7.14e-02  7e-02  8e-02 1:17.5
  800  15200 -3.759499883864592e+07 2.5e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-39185684.928375, sigma=1.10e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=4821)
  ')')


 4900  93100 -3.918568492837461e+07 4.9e+00 8.76e-09  7e-09  2e-08 9:04.6
 5000  95000 -3.918568492837461e+07 4.9e+00 8.59e-09  7e-09  2e-08 9:15.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 5100  96900 -3.918568492837463e+07 4.9e+00 8.23e-09  6e-09  2e-08 9:26.5
 5200  98800 -3.918568492837462e+07 5.0e+00 7.26e-09  6e-09  2e-08 9:37.5
 5300 100700 -3.918568492837462e+07 5.0e+00 7.33e-09  6e-09  2e-08 9:48.5
 5400 102600 -3.918568492837462e+07 5.0e+00 7.13e-09  5e-09  2e-08 9:59.5
 5500 104500 -3.918568492837463e+07 5.0e+00 6.09e-09  5e-09  2e-08 10:10.6
 5600 106400 -3.918568492837462e+07 5.0e+00 5.84e-09  4e-09  2e-08 10:21.7
 5700 108300 -3.918568492837463e+07 5.0e+00 5.68e-09  4e-09  2e-08 10:32.6
 5800 110200 -3.918568492837463e+07 5.0e+00 6.32e-09  5e-09  2e-08 10:43.6
 5900 112100 -3.918568492837463e+07 5.1e+00 6.18e-09  5e-09  2e-08 10:54.7
 6000 114000 -3.918568492837462e+07 5.1e+00 5.90e-09  5e-09  2e-08 11:05.7
Iterat #Fevals   function value  a

In [13]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F9_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:200])
    SVM_Fun [i] = F9(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F9(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([94775.48811092]), count=array([1])) 0.0028575572824007947
SVM
ModeResult(mode=array([40325835.46181241]), count=array([1])) 6820795.347466097
ELN
ModeResult(mode=array([41233491.16840519]), count=array([1])) 8339713.663506051
